In [1]:
from get_financial import *

In [2]:
gicode = '005380'
get_html(gicode)
# get_stock_items(gicode)

20230613_005380_현대차.html file has been created.
현대차:주식 재무 분석 & 적정주가 계산_2023/06/13


'20230613_005380_현대차.html'

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

gicode = '005380'

In [7]:
nv_url = f'https://finance.naver.com/item/main.naver?code={str(gicode)}'
nv_response = requests.get(nv_url)
nv_soup = BeautifulSoup(nv_response.text, 'html.parser')

# NAVER 동일업종 PER 정보
nv_same_per = None
table = nv_soup.find('table', {'summary': '동일업종 PER 정보'})

In [16]:
nv_a = table.find('a')['href']
nv_main = 'https://finance.naver.com'
nv_link = f'{nv_main}{nv_a}'
nv_link

'https://finance.naver.com/sise/sise_group_detail.naver?type=upjong&no=273'